In [1]:
import scipy as sc
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, plot, io
import os
from czifile import imread
import skimage as sk
import pandas as pd


In [2]:
# Define the folder location
folderLoc = os.path.join(os.path.dirname(os.getcwd()), r'Raw_Datasets\Super_Res\2023-06-21')


# Initialize the main dictionary
imageDict = {}
imageDict['Control'] = {}
imageDict['GSK'] = {}

# Get the names of all directories in folderLoc
timepointDirs = [name for name in os.listdir(folderLoc) if os.path.isdir(os.path.join(folderLoc, name)) and name.startswith('TIMEPOINT')]

# Iterate over each timepoint directory
for timepointDir in timepointDirs:
    # Initialize the nested dictionary for each timepoint
    imageDict['Control'][timepointDir] = {}
    imageDict['GSK'][timepointDir] = {}
    
    # Get the paths to each .czi file that begins with C or G
    cziFiles = [os.path.join(folderLoc, timepointDir, file) for file in os.listdir(os.path.join(folderLoc, timepointDir)) if file.startswith(('C', 'G')) and file.endswith('.czi')]

    # Add the paths to the nested dictionary
    for cziFile in cziFiles:
        mainKey = 'Control' if os.path.basename(cziFile).startswith('C') else 'GSK'
        imageDict[mainKey][timepointDir][os.path.basename(cziFile)] = cziFile
    

# Print the resulting dictionary
print(imageDict)


{'Control': {'TIMEPOINT 1 T 0 MIN': {'C1_T1.czi': 'c:\\Users\\Jack\\Documents\\GitHub\\Chromatin_Remodeling_Image_Processing\\Raw_Datasets\\Super_Res\\2023-06-21\\TIMEPOINT 1 T 0 MIN\\C1_T1.czi', 'C2_T1.czi': 'c:\\Users\\Jack\\Documents\\GitHub\\Chromatin_Remodeling_Image_Processing\\Raw_Datasets\\Super_Res\\2023-06-21\\TIMEPOINT 1 T 0 MIN\\C2_T1.czi', 'C3_T1.czi': 'c:\\Users\\Jack\\Documents\\GitHub\\Chromatin_Remodeling_Image_Processing\\Raw_Datasets\\Super_Res\\2023-06-21\\TIMEPOINT 1 T 0 MIN\\C3_T1.czi', 'C4_T1.czi': 'c:\\Users\\Jack\\Documents\\GitHub\\Chromatin_Remodeling_Image_Processing\\Raw_Datasets\\Super_Res\\2023-06-21\\TIMEPOINT 1 T 0 MIN\\C4_T1.czi', 'C5_T1.czi': 'c:\\Users\\Jack\\Documents\\GitHub\\Chromatin_Remodeling_Image_Processing\\Raw_Datasets\\Super_Res\\2023-06-21\\TIMEPOINT 1 T 0 MIN\\C5_T1.czi'}, 'TIMEPOINT 2 T 45 MIN': {'C1_T2.czi': 'c:\\Users\\Jack\\Documents\\GitHub\\Chromatin_Remodeling_Image_Processing\\Raw_Datasets\\Super_Res\\2023-06-21\\TIMEPOINT 2 T 45

In [3]:
def distance(x1, y1, x2, y2):
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def angle(x1, y1, x2, y2):
    return np.arctan2(y2-y1, x2-x1)

In [4]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

def batchProcessing(loc, sample):
    img = imread(loc)

    # Max Projection 
    z_range = np.arange(int(np.round(img.shape[2]/2-5)), int(np.round(img.shape[2]/2+5)))
    img_max = np.max(img[0, 0, z_range, :, :, 0], axis=0) # middle z-slice max projection

    # Filter
    filter_img = sk.filters.gaussian(img_max, sigma=3)

    # Cellpose (makes too round of a mask that does not accurately represent nuclear shape)
    """model = models.Cellpose(model_type='cyto', net_avg=True, gpu=True) # model_type='cyto' or model_type='nuclei'
    mask, flow, style, diams  = model.eval(filter_img, diameter=1000, channels = [0,0])"""

    # Manual Masking
    mask = filter_img > filter_img.flatten().mean()
    mask = sk.morphology.remove_small_objects(mask, min_size=50_000, connectivity=1)
    mask =sk.morphology.remove_small_holes(mask, area_threshold=20_000, connectivity=1)



    labeled_mask = sk.measure.label(mask)
    cell_props = sk.measure.regionprops(labeled_mask)

    roundness = (4*np.pi*cell_props[0].area_filled)/(cell_props[0].perimeter**2)

    # Angle and radius
    perimeter_mask = mask ^ sk.morphology.binary_erosion(mask)

    x, y = np.where(perimeter_mask == 1)
    
    r = np.zeros(len(x))
    theta = np.zeros(len(x))

    for i in range(len(x)):
        r[i] = distance(cell_props[0].centroid[0], cell_props[0].centroid[0], x[i], y[i])
        theta[i] = angle(cell_props[0].centroid[0], cell_props[0].centroid[0], x[i], y[i])

    theta, r = zip(*sorted(zip(theta,r))) # order them by angle



    # Save plots to PDF
    with PdfPages(f'{sample}.pdf') as pdf:
        plt.imshow(np.array(img[0, 0, int(np.round(img.shape[2]/2)), :, :, 0]))
        plt.title('Original Image')
        pdf.savefig()
        plt.close()

        plt.imshow(img_max)
        plt.title('Max Projection')
        pdf.savefig()
        plt.close()

        plt.imshow(filter_img)
        plt.title('Filtered Image')
        pdf.savefig()
        plt.close()

        plt.imshow(mask)
        plt.title('Mask')
        pdf.savefig()
        plt.text(1,1, f'Roundness: {roundness}, Area: {cell_props[0].area_filled}px^2, Perimeter: {cell_props[0].perimeter}px')
        plt.close()

        plt.imshow(perimeter_mask)
        plt.title('Perimeter')
        pdf.savefig()
        plt.close()

        plt.plot(theta, r)
        plt.xlabel('Radians')
        plt.ylabel('Radius from centroid')
        pdf.savefig()
        plt.close()

        '''fig = plt.figure(figsize=(12,5))
        plt.title('Cellpose Segmentation')
        plot.show_segmentation(fig, img_max, mask, flow[0], channels=[0,0])
        plt.tight_layout()
        pdf.savefig()
        plt.close()'''
    


    # returns area, centroid, equivalent_diameter, major_axis_length, minor_axis_length, perimeter, solidity, eccentricity
    return cell_props[0].area_filled, cell_props[0].centroid, cell_props[0].equivalent_diameter, cell_props[0].major_axis_length, cell_props[0].minor_axis_length, cell_props[0].perimeter, cell_props[0].solidity, cell_props[0].eccentricity, roundness, theta, r
    

In [5]:
outputdf = pd.DataFrame(columns = ['Treatment', 'Timepoint', 'Sample', 'Area', 'Centroid', 'Equivalent Diameter', 
                                   'Major Axis Length', 'Minor Axis Length', 'Perimeter', 'Solidity', 'Eccentricity', 'Roundness'])

for treatment in imageDict.keys():
    for timepoint in imageDict[treatment].keys():
        for sample in imageDict[treatment][timepoint].keys():
            area, centroid, equivalent_diameter, major_axis_length, minor_axis_length, perimeter, solidity, eccentricity, roundness, theta, radius = batchProcessing(imageDict[treatment][timepoint][sample], sample)
            outputdf = outputdf.append({'Treatment': treatment, 'Timepoint': timepoint, 'Sample': sample, 'Area': area, 'Centroid': centroid, 'Equivalent Diameter': equivalent_diameter, 'Major Axis Length': major_axis_length, 'Minor Axis Length': minor_axis_length, 
                                        'Perimeter': perimeter, 'Solidity': solidity, 'Eccentricity': eccentricity, 'Roundness': roundness}, ignore_index=True)
            np.save(f'theta_{sample}.npy', theta)
            np.save(f'radius_{sample}.npy', radius)




In [6]:
outputdf.to_csv('output.csv', index=False)

In [7]:
outputdf

,Treatment,Timepoint,Sample,Area,Centroid,Equivalent Diameter,Major Axis Length,Minor Axis Length,Perimeter,Solidity,Eccentricity,Roundness
0,Control,TIMEPOINT 1 T 0 MIN,C1_T1.czi,191063.0,"(551.1302292960961, 693.6928814056097)",493.223040,620.828111,392.517271,1795.866233,0.983163,0.774766,0.744455
1,Control,TIMEPOINT 1 T 0 MIN,C2_T1.czi,173080.0,"(526.78636468685, 726.1338456205223)",469.438282,677.243581,326.036144,1736.920056,0.987004,0.876492,0.720937
2,Control,TIMEPOINT 1 T 0 MIN,C3_T1.czi,139314.0,"(666.6038660866819, 688.3021447952108)",421.165162,651.291787,275.312785,1689.866233,0.949762,0.906261,0.613056
3,Control,TIMEPOINT 1 T 0 MIN,C4_T1.czi,197740.0,"(585.9283301304744, 577.6294073025184)",501.767264,696.043710,362.769212,1971.856131,0.969356,0.853442,0.639078
4,Control,TIMEPOINT 1 T 0 MIN,C5_T1.czi,202101.0,"(586.5242230369963, 674.3319033552531)",507.270130,599.565050,429.649933,1769.238816,0.987453,0.697482,0.811345
5,Control,TIMEPOINT 2 T 45 MIN,C1_T2.czi,164326.0,"(606.9042208780107, 554.144937502282)",457.412682,608.161395,345.313798,1634.937300,0.986884,0.823167,0.772527
6,Control,TIMEPOINT 2 T 45 MIN,C2_T2.czi,180682.0,"(612.62717924309, 582.0995782645754)",479.636808,686.707375,335.581275,1749.304833,0.992093,0.872462,0.741983
7,Control,TIMEPOINT 2 T 45 MIN,C3_T2.czi,140987.0,"(646.9451793427763, 667.3287749934391)",423.686469,659.536037,277.244639,1913.044948,0.909365,0.907356,0.484104
8,Control,TIMEPOINT 2 T 45 MIN,C4_T2.czi,175559.0,"(636.2922094566499, 673.9550407555295)",472.788178,637.045691,352.031678,1940.861182,0.956667,0.833446,0.585658
9,Control,TIMEPOINT 2 T 45 MIN,C5_T2.czi,212981.0,"(628.8230593339312, 605.6228255102568)",520.745457,605.151016,448.540078,1798.108873,0.988183,0.671280,0.827787
